In [ ]:
#우선 Link prediction의 정의부터 다시 잡고 넘어가자.
#왜 node classification에 대한 정의는 생략했냐면, 석사과정 연구를 모두 다 node classification에 대해서 진행했고 머릿속에 개념이 잘 잡혀있어서 생략하고 넘어갔다.
#하지만 link prediction은 예전에 공부하고 까먹었기 때문에 다시 한번 잡고 이어서 heterogeneous graph에서 link prediction이 어떻게 진행되는지 알아보자

#Link prediction의 개념 
#링크 예측은 현재 네트워크(graph)에서 나타나지 않은 연결을 예측하거나 미래의 네트워크에서 새롭게 생겨나거나 없어지는 것을 예측하는 것이다.  
#일반적으로는 공통된 이웃(common neighbor nodes)의 수와 같은 그래프 지표를 활용해서 관측되지 않은 링크를 찾아낸다.
#그러면 일반적인 인공지능 모델들과 비슷하게 train , valid , test set으로 나눠야하는데 link를 예측하는것이므로 일종의 subgraph가 들어간다 라고 생각하면 된다. (train edge로만 이루어진 그래프로 학습을 진행하고 test edge에 대한 링크를 추론해봄)
#또한 positive, negative 로 나누는데, 이는 positive는 연결되어있는데 추후에 어떻게 될것인지에 대한 추측을 하기 위해서 연결되어있는것들, negative는 연결이 안되어있는데 추후에 어떻게 될것인지에 대한 추측을 위해 연결이 안되어있는 쌍들을 나타낸다. 
#이 두가지를 고려해서 좀 더 좋은 성능을 보인다고 한다.
#우선 데이터셋은 임의로 만들어보자

In [1]:
import torch
import numpy as np
import dgl
import dgl.data

dataset = dgl.data.CoraGraphDataset()

g = dataset[0]

src,dst = g.edges()
print(src,dst)

Extracting file to C:\Users\HOME\.dgl\cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
tensor([   0,    0,    0,  ..., 2707, 2707, 2707]) tensor([ 633, 1862, 2582,  ...,  598, 1473, 2706])


In [5]:
#train test 나눠줌 예제라서 validation은 추가하지 않겠음
eids = np.arange(g.num_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids)*0.1) #1055개를 test set으로 사용

train_size = g.num_edges() - test_size
test_pos_src, test_pos_dst = src[eids[:test_size]], dst[eids[:test_size]]
train_pos_src, train_pos_dst = src[eids[test_size:]], dst[eids[test_size:]]


In [14]:
#위에서 test와 train을 구분해주었고 이제 negative sampling을 해준다
import scipy.sparse as sp

adj = sp.coo_matrix((np.ones(len(src)),(src.numpy(),dst.numpy())))
#coo_matrix(data,(row,col)) row와 col에 동시에 나타나면 1로 만들어 인접행렬 생성
adj_neg = 1- adj.todense()
neg_src, neg_dst = np.where(adj_neg !=0) #연결 안되어있는 노드 쌍 추출

#negative sampling 숫자는 많을 수 밖에 없음 연결되어있지 않은 모든 상황을 고려하기 때문에
neg_eids = np.random.choice(len(neg_src),g.num_edges()//2)
test_neg_u, test_neg_v = neg_src[neg_eids[:test_size]], neg_dst[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_src[neg_eids[test_size:]], neg_dst[neg_eids[test_size:]]

train_g = dgl.remove_edges(g, eids[:test_size])
print(train_g)

Graph(num_nodes=2708, num_edges=9501,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={})


In [32]:
from dgl.nn import SAGEConv
import torch.nn.functional as F
import dgl.function as fn
import torch.nn as nn
#graph sage를 사용해서 구축할 예정임.
#graphsage가 inductive한 환경에서 실행될 수 있기도하고 빠르기도 하기 때문에 baseline으로 설정함

class GraphSAGE(nn.Module):
    def __init__(self,in_feats,hidden_feats):
        super(GraphSAGE,self).__init__()

        self.in_feats = in_feats
        self.hidden_feats = hidden_feats

        self.sage1 = SAGEConv(in_feats, hidden_feats, aggregator_type = 'mean')
        self.sage2 = SAGEConv(hidden_feats,hidden_feats,aggregator_type='mean')
    def forward(self,g,features):
        h = self.sage1(g,features)
        h = F.relu(h)
        h = self.sage2(g,h)
        return h
    
class DotPredictor(nn.Module):
    def forward(self,g,features):
        with g.local_scope():
            g.ndata['feat'] = features
            g.apply_edges(fn.u_dot_v('feat','feat','score'))
            return g.edata['score']

In [34]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)

pred = DotPredictor()
import torch.optim as optim
from sklearn.metrics import roc_auc_score
def criterion(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    labels = labels.unsqueeze(1)
    return F.binary_cross_entropy_with_logits(scores, labels)

def accuracy(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    #labels = labels.unsqueeze(1)

    return roc_auc_score(labels, scores)
    
all_logits = []
train_pos_g = dgl.graph((train_pos_src,train_pos_dst))
train_neg_g = dgl.graph((train_neg_u,train_neg_v))
optimizer = optim.Adam(model.parameters(),lr = 0.01)
for epoch in range(50):
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = criterion(pos_score, neg_score)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0 :
        print(f'In epoch {epoch+1}, loss: {loss:.4f}')
test_pos_g = dgl.graph((test_pos_src,test_pos_dst))
test_neg_g = dgl.graph((test_neg_u,test_neg_v))
h = model(train_g,train_g.ndata['feat'])
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', accuracy(pos_score, neg_score))

In epoch 5, loss: 0.6116
In epoch 10, loss: 0.5949
In epoch 15, loss: 0.5647
In epoch 20, loss: 0.5188
In epoch 25, loss: 0.4516
In epoch 30, loss: 0.3904
In epoch 35, loss: 0.3636
In epoch 40, loss: 0.3268
In epoch 45, loss: 0.2971
In epoch 50, loss: 0.2732
AUC 0.8763244311673144
